# Bob

* Bob wants to receive a message from Alice.
* He can communicate with Alice over an unsecure, classical channel.
* They can also communicate over a quantum channel.

Bob establishes a classical channel to Alice. Anyone can listen in:

# setup

* wait for setup data from Alice

In [ ]:
import paho.mqtt.client as mqtt
import json

mqtt_broker_ip = "broker.emqx.io"
channel = "quantum_cryptography/classical_channel/#"

client = mqtt.Client()
client.connect(mqtt_broker_ip, 1883, 60)
# How many bits
setup_data = None


def on_message_setup(client, userdata, msg):
    global setup_data
    try:
        setup_data = json.loads(msg.payload)
        print("Bits to Exchange:", setup_data["bits_to_exchange"])
        client.disconnect()
    except Exception as e:
        print("unable to parse json: ", msg.payload, "\n", e)


def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(channel)


client.on_message = on_message_setup
client.on_connect = on_connect

_ = client.loop_forever()

## create list of random polarizations

In [ ]:
from enum import Enum
from random import choice


class Basis(Enum):
    # 0° | or _ 90°:
    HV = 0
    # -45° \ or 45° /:
    LR = 1


polarizations = list()

for i in range(setup_data["bits_to_exchange"]):
    polarizations.append(choice(list(Basis)))

for i, b in enumerate(polarizations):
    print(i, b, sep="\t")

# quantum channel

In [ ]:
from time import sleep
from qcrypt.hardware.filterwheel import Filterwheel
from qcrypt.hardware.lightsensor import Lightsensor
import paho.mqtt.client as mqtt

broker = "broker.emqx.io"
channel_laser = "quantum_cryptography/classical_channel/alice/laser"

client = mqtt.Client()
filterwheel = Filterwheel(username="bob", mqtt_broker_ip=broker)
lightsensor = Lightsensor()


def on_message_quantum(client, userdata, msg):
    print(msg.payload.decode())
    if msg.payload.decode() == "on":
        # Sensor starten - 1 Wertepaar ausgeben
        # print(lightsensor.getValuesArray())
        print(lightsensor.getHighLow())
        # sensor starten

client.connect(broker, 1883, 60)
client.subscribe(channel)
client.add_message_callback = message_callback_add(channel_laser,on_message_quantum)
client.loop_forever()